In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import ta

In [ ]:
data = pd.read_csv(
    'data/histdata/HISTDATA_COM_ASCII_EURUSD_M1_2023/DAT_ASCII_EURUSD_M1_2023.csv',
    header=None,
    index_col='timestamp',
    delimiter=';',
    names=['timestamp', 'open', 'high', 'low', 'close', 'volume'],
    parse_dates=['timestamp'],
    date_format='%Y%m%d %H%M%S'
)

In [ ]:
def show_day_chart(df, date_str):
    # 指定日のデータを抽出
    target_date = pd.Timestamp(date_str).date()
    day_data = df[df.index.date == target_date]
    
    if day_data.empty:
        print(f"No data found for {date_str}")
        return
    
    # 15分足にリサンプリング
    day_data_15m = day_data.resample('15min').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()
    
    if day_data_15m.empty:
        print(f"No resampled data available for {date_str}")
        return
    
    # ろうそく足チャートを表示
    mpf.plot(day_data_15m, 
             type='candle',
             style='charles',
             title=f'EUR/USD {date_str} (15min)',
             ylabel='Price',
             figsize=(12, 8))
    
    plt.show()

In [ ]:
show_day_chart(data, '2023-01-01')

In [ ]:
def get_resampled_data(df, date_str, timeframe='15min'):
    """
    指定した日付のデータを指定した時間足でリサンプリングして返す
    
    Parameters:
    date_str (str): 対象日付 (例: '2023-01-01')
    timeframe (str): リサンプリング間隔 (例: '15min', '1h', '4h', '1d')
    
    Returns:
    pd.DataFrame: リサンプリングされたOHLCVデータ
    """
    # 指定日のデータを抽出
    target_date = pd.Timestamp(date_str).date()
    day_data = df[df.index.date == target_date]
    
    if day_data.empty:
        print(f"No data found for {date_str}")
        return pd.DataFrame()
    
    # 指定した時間足でリサンプリング
    resampled_data = day_data.resample(timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()
    
    if resampled_data.empty:
        print(f"No resampled data available for {date_str} with {timeframe} timeframe")
        return pd.DataFrame()
    
    return resampled_data

In [ ]:
df = get_resampled_data(data, '2023-01-01', '15min')
df.head(20)

In [ ]:
df['sma5'] = ta.trend.SMAIndicator(df.close, 5).sma_indicator()
df.head(20)